<div style="direction:rtl;">
    <h1>  گزارش بخش org_extractor:</h1>
</div>

<div style="direction:rtl;">
 هدف از انجام این تسک، استخراج نام تمامی سازمان های موجود در متن ورودی است. به همین جهت مجموعه ی نسبتا مفصلی از نام سازمان های کشور تهیه گردید. 
 </div>

<div style="direction:rtl;"> 
در ابتدا، برای استفاده از منابعی مانند مجموعه ی نام سازمان ها و مدل های هضم به روش زیر، منابع لازم را فراهم می کنیم.
</div>

In [1]:
from hazm import * 
import re

DICT_PATH = 'resource/org/orgs.txt'
TAGGER_PATH = 'resource/hazm_model/postagger.model'
CHUNKER_PATH = 'resource/hazm_model/chunker.model'
ROLES = ['NP', 'PP', 'VP', 'ADJP', 'ADVP']

content = open(DICT_PATH, 'r+').readlines()
normalizer = Normalizer()
tagger = POSTagger(model=TAGGER_PATH)
chunker = Chunker(model=CHUNKER_PATH)

ValueError: Invalid model file 'resource/hazm_model/postagger.model'

<div style="direction:rtl;"> 
جهت ساختاربندی مناسب، از کلاس org_extractor استفاده می کنیم.
</div>

In [2]:
class org_extractor:
    
    def __init__(self, text) -> None:
        self.text = text


<div style="direction:rtl;"> 
در اولین گام راه حل، به کمک
chunker
کتابخانه ی هضم متن را چانک بندی می کنیم.
سپس کاراکترهای زائد متن که به حل مسئله کمکی نمی کنند را حذف کرده و چانک ها را برای مرحله ی بعد آماده می کنیم.<br/>
* بخش آخر به دلیل مدیریت کردن اشتباهات در چانک ها پس از تبدیل به فرم مورد نیاز اضافه شده
</div>

In [3]:
    def chunk_text(self):
        self.text = self.normalizer.normalize(self.text)
        tagged = self.tagger.tag(word_tokenize(self.text))
        chunks = tree2brackets(self.chunker.parse(tagged))

        chunked = [[c.strip(), c.split()[-1]] for c in\
                    [re.sub(r'\.|،|]*|,|;|/|[۰-۹]+|\)|\(|-|«|»', '', chunk).strip() for chunk in chunks.split('[')] if len(c.split()) >= 2]

        for c in chunked:
            c[0] = c[0].replace('\u200c', ' ')

        roles_set = set(self.ROLES)

        for chunk in chunked:
            words = chunk[0].split()
            filtered_words = [word for word in words if word not in roles_set]

            chunk[0] = ' '.join(filtered_words)

            if chunk[1] not in roles_set:
                for word in words:
                    if word in roles_set:
                        chunk[1] = word 
                        break  

        return chunked

NameError: name 'self' is not defined

<div style="direction:rtl;"> 
می دانیم نام سازمان ها در یک چانک NP قرار می گیرد. اما گاهی این نام به دلیل طولانی بودن شکسته شده و در چند چانک متوالی NP جایگیری می کند. به همین دلیل تمامی چانک های NP متوالی را ادغام میکنیم و صرفا آنها را در نظر میگیریم.
</div>

In [4]:
    def merge_np_chunks(self, chunked):
        nps, temp, id = [], '', 0

        while id < len(chunked):
            chunk = chunked[id]
            if chunk[1]=='NP':
                temp = chunk[0]
                while id<len(chunked)-1 and chunked[id+1][1]=='NP':
                    temp = temp + ' ' + chunked[id+1][0]
                    id = id + 1
                    
                else:
                    nps.append(temp)
                    temp = ''
            id= id+1

        return nps

<div style="direction:rtl;"> 
در ادامه برای هر چانک از تابع ngram استفاده می کنیم. مقدار n  در هر یک از این ngramها به ترتیب از تعداد کلمات هر چانک به یک کلمه کاهش می یابند.<br/>
دلیل این کار آن است که به ترتیب از بزرگترین عبارت در مجموعه ی نام سازمان ها جستجو شود.
</div>

In [5]:
def make_ngrams(self, nps):
    ngs = []
    for np in nps:
        npg = []
        count = len(np.split())
        for i in range(count, 0, -1):
            npg.extend(self.ngrams(np, i))
        ngs.append(npg)

    return ngs

<div style="direction:rtl;"> 
سپس به کمک عبارات قاعده محور،‌ ngramهای ایجاد شده را در مجموعه ی نام سازمانها جستجو میکنیم.
خروجی لیستی از تاپل هاست که در هر تاپل نام سازمان و اندیس شروع و پایان آن در جمله قرار دارد. 
</div>

In [6]:
def find_org(self):

        chunked = self.chunk_text()
        nps = self.merge_np_chunks(chunked=chunked)
        ngs = self.make_ngrams(nps=nps)
        
        output = []
        for row in ngs:
            t = []
            for ng in row: 
                if len(ng) < 4:
                    continue
                for org in self.content:
                    otp = re.match(f'^{ng}.*', org)

                    if isinstance(otp, re.Match):
                        t.append((otp))
            
            output.append(t)

        result = []
        for ng, otp in zip(ngs, output):
            for o in otp:
                mtch = o.group(0)
                if mtch in ng and mtch not in result:
                    result.append(mtch)
        
        result = sorted(result, key=len, reverse=True)
        defined_terms = []

        for term in result:
            if not any(term in other for other in defined_terms):
                for m in re.finditer(term, self.text):
                    defined_terms.append((term, (m.start(), m.end())))
        
        return defined_terms
